In [43]:
!pip install -U sklearn

Requirement already up-to-date: sklearn in /usr/local/lib/python3.7/dist-packages (0.0)


In [44]:
import pandas as pd
import numpy as np
import csv

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

data = pd.read_csv('reviews.csv', delimiter='\t')

In [45]:
data.head()

,Skype,3,2021-01-12T17:06:39+00:00,It shows my credit in Danish Corona but I live in Finland. I don´t see how to change currency.
0,Skype,1,2021-01-10T16:58:00+00:00,Disgusting. Skype effectively operated as a sc...
1,Skype,3,2020-12-28T11:57:59+00:00,"Outdated.\n\n Simply an outdated platform, it ..."
2,Skype,1,2020-12-27T14:29:41+00:00,When Skype came out I thought it was a great a...
3,Skype,5,2020-12-20T21:05:07+00:00,My favorite app for business calls.
4,Skype,4,2020-12-18T21:49:13+00:00,"I use Skype everyday for work, and I must admi..."


In [46]:
data.columns = ['company', 'rating', 'date', 'review']

In [47]:
data['Sentiment'] = np.zeros(data.shape[0])
for i in range(data.shape[0]):
  if data.rating[i] <=2:
    data['Sentiment'][i] = 0;
  elif data.rating[i] == 3:
    data['Sentiment'][i] = 1;
  else:
    data['Sentiment'][i] = 2
data['Sentiment'] = data['Sentiment'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
data.dtypes

company      object
rating        int64
date         object
review       object
Sentiment     int64
dtype: object

In [49]:
data.groupby('Sentiment').count()

,company,rating,date,review
Sentiment,,,,
0,832,832,832,832
1,72,72,72,72
2,387,387,387,387


In [50]:
data.head()

,company,rating,date,review,Sentiment
0,Skype,1,2021-01-10T16:58:00+00:00,Disgusting. Skype effectively operated as a sc...,0
1,Skype,3,2020-12-28T11:57:59+00:00,"Outdated.\n\n Simply an outdated platform, it ...",1
2,Skype,1,2020-12-27T14:29:41+00:00,When Skype came out I thought it was a great a...,0
3,Skype,5,2020-12-20T21:05:07+00:00,My favorite app for business calls.,2
4,Skype,4,2020-12-18T21:49:13+00:00,"I use Skype everyday for work, and I must admi...",2


In [51]:
# drop extra reviews to gain a balance dataset
# separate dataset by sentiment and down sample extra reviews
df_0 = data[data['Sentiment'] ==0]
df_1 = data[data['Sentiment'] ==1]
df_2 = data[data['Sentiment'] ==2]

df_0_down = resample(df_0, replace = False, n_samples = len(df_1), random_state = 0)
df_2_down = resample(df_2, replace = False, n_samples = len(df_1), random_state = 1)

data_down = pd.concat([df_0_down, df_1, df_2_down])
data_down = data_down.drop(columns=['company', 'rating', 'date'])

In [52]:
data_down.sample(10)

,review,Sentiment
1193,"Outdated.\n\n Simply an outdated platform, it ...",1
923,"The sound quality is quite bad, impossible to ...",0
811,So far nothing beats SKYPE.,2
822,Traveling abroad was always a challenge when i...,2
1019,I long for the days when Skype had a simple us...,1
33,"The app itself is very good, but the service c...",2
980,We used to phone the USA and had a deal with B...,2
546,What absolute junk the new Skype is for Mac.\n...,0
1047,there is better message systems than this.. wr...,1
1093,"Outdated.\n\n Simply an outdated platform, it ...",1


In [53]:
data_down.groupby('Sentiment').count()

,review
Sentiment,
0,72
1,72
2,72


In [54]:
train, valid = train_test_split(data_down, test_size=0.33, shuffle=True, random_state=111)
train.to_csv('train.csv',index=False)
valid.to_csv('valid.csv',index=False)

In [55]:
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')

In [56]:
print("\n".join(train['review'][0].split("\n")))

I Chose Skype because I can communicate with distant relatives


In [57]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train['review'])
X_train_counts.shape

(144, 1453)

In [58]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(144, 1453)

In [59]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train['Sentiment'])

In [60]:
docs_new = ['I love it', 'Worst forever']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, category))

'I love it' => 2
'Worst forever' => 0


In [61]:
from sklearn.pipeline import Pipeline
text_clf_MNB = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [62]:
text_clf_MNB.fit(train['review'], train['Sentiment'])

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [63]:
docs_valid = valid['review']
predicted = text_clf_MNB.predict(docs_valid)
acc_MNB = np.mean(predicted == valid['Sentiment'])

print('Accuracy score: ', acc_MNB)

Accuracy score:  0.7083333333333334


In [64]:
from sklearn.linear_model import SGDClassifier
text_clf_SGD = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=20, tol=None)),
])

text_clf_SGD.fit(train['review'], train['Sentiment'])

predicted = text_clf_SGD.predict(docs_valid)
acc_SGD = np.mean(predicted == valid['Sentiment'])

print('Accuracy score: ', acc_SGD)

Accuracy score:  0.7638888888888888


In [65]:
from sklearn import metrics
print(metrics.classification_report(valid['Sentiment'], predicted))

              precision    recall  f1-score   support

           0       0.84      0.75      0.79        28
           1       0.62      0.68      0.65        22
           2       0.83      0.86      0.84        22

    accuracy                           0.76        72
   macro avg       0.76      0.77      0.76        72
weighted avg       0.77      0.76      0.77        72



In [75]:
f1 = metrics.f1_score(valid['Sentiment'], predicted, average= None)
cmtx = pd.DataFrame(
    metrics.confusion_matrix(valid['Sentiment'], predicted), 
    index=['negative', 'neutral', 'positive'], 
    columns=['negative', 'neutral', 'positive'])


In [91]:
print('Accuracy score: ', acc_SGD, end='')
print("\n")
print('F1_score for negative rating: ', f1[0])
print('F1_score for neutral rating: ', f1[1])
print('F1_score for positive rating: ', f1[2])
print("\n")
print('confusion matrix: \n',cmtx)

Accuracy score:  0.7638888888888888

F1_score for negative rating:  0.7924528301886793
F1_score for neutral rating:  0.6521739130434783
F1_score for positive rating:  0.8444444444444444


confusion matrix: 
           negative  neutral  positive
negative        21        6         1
neutral          4       15         3
positive         0        3        19
